https://nbviewer.jupyter.org/github/zhedongzheng/finch/blob/master/finch/tensorflow2/spoken_language_understanding/atis/main/bigru.ipynb

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/combo/PTCombo/'

COMBO2.drawio  NER  PTCombo.ipynb  PTCombo.py  UD  UD.tar.gz  utils.py


In [ ]:
!ls '/content/drive/My Drive/combo/PTCombo/UD/PL/pl_sz-ud-train.conllu'

'/content/drive/My Drive/combo/PTCombo/UD/PL/pl_sz-ud-train.conllu'


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [4]:
!pip install bpemb==0.3.0
!pip install conllu==2.2
!pip install --upgrade tensorflow_addons

     |████████████████████████████████| 1.1MB 11.0MB/s 
     |████████████████████████████████| 1.1MB 9.0MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


https://nlp.h-its.org/bpemb/

In [6]:
from bpemb import BPEmb
from conllu import parse_incr
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
import tensorflow_addons as tfa
import pprint 
import time, sys, logging


print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())
if tf.test.is_gpu_available():
    print(tf.test.gpu_device_name())
else:
    print("TF cannot find GPU")


params = {
  'train_path_ud': "/content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu",
  'test_path_ud': "/content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-test.conllu",
  'train_path_ner': "/content/drive/My Drive/combo/PTCombo/NER/nkjp-ner-train.conllu",
  'test_path_ner': "/content/drive/My Drive/combo/PTCombo/NER/nkjp-ner-test.conllu",
  'batch_size': 16,
  'num_samples': 4978,
  'rnn_units': 100,
  'dropout_rate': 0.2,
  'clip_norm': 5.0,
  'lr': 3e-4,
  'num_patience': 3,
  'pos_size': 100,
  'heads_size': 100,
  'token_len': 6,
  'sentence_len': 67,
  'embedding_size': 100,
  'vocabulary_size': 100000,
  'debug': False
}

print('Loading BPEmb')
bpemb_pl = BPEmb(lang="pl", dim=params["embedding_size"], vs=params["vocabulary_size"])


TensorFlow Version 2.2.0
GPU Enabled: True
/device:GPU:0
Loading BPEmb


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17639733404970794261, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10816129171024996004
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3620361946992980852
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12924993661078217930
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
ids = bpemb_pl.encode_ids('Ala ma kota')
print(ids)
vecs = bpemb_pl.vectors[ids]
print(vecs)
#print(np.mean(vecs,axis=0).shape)
#bpemb_pl.vectors.shape
#>>> ids = bpemb_zh.encode_ids("ala")
#[25950, 695, 20199]
#>>> bpemb_zh.vectors[ids].shape
#(3, 100)

[5695, 123, 24167]
[[ 0.138254 -0.266206  0.094003  0.110085 -0.350992  0.629002  0.047738
  -0.229923  0.106885  0.100686 -0.409646  0.236586  0.102575 -0.113163
  -0.095766 -0.152248 -0.357291  0.315834  0.001846 -0.184776 -0.070891
  -0.434788 -0.884103 -0.458626  0.101739  0.227307 -0.19136   0.143326
  -0.216715 -0.405611 -0.181386  0.00354   0.312164  0.17422  -0.309512
  -0.315861 -0.002145 -0.532517 -0.855398 -0.07617  -0.759839  0.028048
   0.318065 -0.247331  0.079041 -0.179897  0.138402  0.310474  0.054178
  -0.131207 -0.055599 -0.385557 -0.37365   0.599516  0.936611 -0.099981
   0.278162 -0.255666  0.215046  0.193898 -0.29795  -0.132868 -0.122183
   0.58161  -0.305831 -0.363126  0.458534 -0.582399 -0.633643  0.029885
  -0.269724  0.079161 -0.675176 -0.253526 -0.304391  0.512647 -0.406768
   0.190714  0.424885 -0.334016 -0.431855  0.338523 -0.420212  0.192451
  -0.296747  0.354132  0.667544  0.737998  0.181646 -0.568619  0.435191
  -0.054256  0.155529  0.267217  0.124412 -0.

In [ ]:
tokenlist[0]

OrderedDict([('id', 1),
             ('form', 'We'),
             ('lemma', 'w'),
             ('upostag', 'ADP'),
             ('xpostag', 'prep:acc:wok'),
             ('feats',
              OrderedDict([('AdpType', 'Prep'),
                           ('Case', 'Acc'),
                           ('Variant', 'Long')])),
             ('head', 2),
             ('deprel', 'case'),
             ('deps', None),
             ('misc', None)])

Inicjalizacja oraz uruchomienie label encoderow:

In [8]:
le_upostag = preprocessing.LabelEncoder()
le_deprel = preprocessing.LabelEncoder()

# this method iterates CONLL files and outputs relevant fields 
# to generate Label Encoder IDs and compute various stats
def data_generator_tokens(f_path, params, bpemb_pl):
  print('Reading', f_path)
  data_file = open(f_path, "r", encoding="utf-8")
  for tokenlist in parse_incr(data_file):

      orths = [ t['form'] for t in tokenlist ] 
      lemmas = [ t['lemma'] for t in tokenlist ]
      upostags = [ t['upostag'] for t in tokenlist ]      
      heads = [ t['head'] for t in tokenlist ]
      deprels = [ t['deprel'] for t in tokenlist ]

      assert len(orths) == len(lemmas) == len(upostags) == len(heads) == len(deprels)

      # subword-level collections:
      sub_tokenids = []
      sub_upostags = []
      sub_heads = []
      sub_deprels = []

      for nr, orth in enumerate(orths):
        tokenids = bpemb_pl.encode_ids(orth)
        sub_tokenids.extend( tokenids )
        sub_upostags.extend( [upostags[nr]] * len(tokenids) )
        sub_heads.extend( [heads[nr]] * len(tokenids) )
        sub_deprels.extend( [deprels[nr]] * len(tokenids) )

      yield (sub_tokenids, (sub_upostags, sub_heads, sub_deprels))

toksizes = [] ; postags_all = [] ; deprels_all = [] ; sentence_lens = []
nr = 0
for conll_tuple in data_generator_tokens(params["train_path_ud"], params, bpemb_pl):
  sub_tokenids, (upostags, heads, deprels) = conll_tuple[:]
  sentence_lens.append( len(sub_tokenids) )
  postags_all.extend(upostags)
  deprels_all.extend(deprels)

print("subword token stats:")
print(np.unique(toksizes, return_counts=True))
print("sentence length stats:")
print(np.unique(sentence_lens, return_counts=True))

le_upostag.fit(postags_all)
le_deprel.fit(deprels_all)


Reading /content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu
subword token stats:
(array([], dtype=float64), array([], dtype=int64))
sentence length stats:
(array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  82,  83,  84,  90,  91,  94,  95,  96,  97, 107, 113, 117]), array([ 10,  87, 244, 387, 470, 558, 717, 827, 903, 935, 963, 891, 887,
       785, 793, 735, 621, 625, 508, 448, 436, 397, 401, 349, 322, 285,
       283, 228, 210, 190, 187, 171, 185, 145, 139, 126, 138, 106, 100,
        76,  72,  70,  74,  75,  64,  47,  36,  43,  42,  38,  35,  18,
        25,  12,  16,  12,  19,  19

LabelEncoder()

In [ ]:
le_upostag

In [9]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [10]:
# this method iterates CONLL files and outputs padded IDs  
# it's intended as TensorFlow graph input
def data_generator_ids_UD(f_path, params, bpemb_pl, le_upostag, le_deprel):
  print('Reading', f_path)
  data_file = open(f_path, "r", encoding="utf-8")
  for tokenlist in parse_incr(data_file):

      orths = [ t['form'] for t in tokenlist ]
      lemmas = [ t['lemma'] for t in tokenlist ]
      upostags = [ t['upostag'] for t in tokenlist ]
      heads = [ t['head'] for t in tokenlist ]
      deprels = [ t['deprel'] for t in tokenlist ]

      assert len(orths) == len(lemmas) == len(upostags) == len(heads) == len(deprels)

      #zeros = 0
      #for h in heads:
      #  if h == 0: zeros += 1
      #print("zeros: "+str(zeros))

      # fix broken data:
      heads = [ 0 if h is None else h for h in heads]
      heads = [ 0 if h >=params["sentence_len"] else h for h in heads]

      # 
      #fixed_len = 0
      #for h in heads:
      #  if h >=params["sentence_len"]: fixed_len += 1
      #print("zeros: "+str(fixed_len))


      # subword-level collections:
      sub_tokenids = []
      sub_upostags = []
      sub_heads = []
      sub_deprels = []

      for nr, orth in enumerate(orths):
        tokenids = bpemb_pl.encode_ids(orth)
        sub_tokenids.extend( tokenids )
        sub_upostags.extend( [upostags[nr]] * len(tokenids) )
        sub_heads.extend( [heads[nr]] * len(tokenids) )
        sub_deprels.extend( [deprels[nr]] * len(tokenids) )
      
      sub_upostags_ids = le_upostag.transform(sub_upostags)
      sub_deprels_ids = le_deprel.transform(sub_deprels)

      yield (sub_tokenids, (sub_upostags_ids, sub_heads, sub_deprels_ids ))

def dataset_UD(is_training, params, bpemb_pl, le_upostag, le_deprel):

  _shapes = ( (None,), ((None,), (None,), (None,)))
  _types = (tf.int32, (tf.int32, tf.int32, tf.int32))
  _pads = (0, (0, 0, 0)) # no neg values here!
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_UD(params['train_path_ud'], params, bpemb_pl, le_upostag, le_deprel),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_UD(params['test_path_ud'], params, bpemb_pl, le_upostag, le_deprel),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [11]:
class Model(tf.keras.Model):

  def __init__(self, params: dict, embedding_matrix):
    super().__init__()

    #self.masking = tf.keras.layers.Masking(mask_value=-1) # not working!
    
    self.embeddings = tf.keras.layers.Embedding(params['vocabulary_size'], params['embedding_size'], 
                                                embeddings_initializer=tf.keras.initializers.Constant(bpemb_pl.vectors), 
                                                trainable=False, mask_zero=True)

    #self.embedding = tf.Variable(embedding_matrix, dtype=tf.float32, name='pretrained_embedding', trainable=False)
    
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.bidir_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['rnn_units'], return_sequences=True))
    # orths, lemmas, (upostags, heads, deprels)
    #self.out_linear_ner = tf.keras.layers.Dense(len(le_ner.classes_) , name='output_ner')

    # part-of-speech:
    self.out_linear_pos = tf.keras.layers.Dense(len(le_upostag.classes_) , name='output_pos')

    # dependency parser dependencies:
    # TODO -- heads need N+1 classes, the extra last class is for None entries in CONLLU:    
    self.linear_dep_arc = tf.keras.layers.Dense(params["sentence_len"]+2, name='dep_arc')
    self.linear_head_arc = tf.keras.layers.Dense(params["sentence_len"]+2, name='head_arc')
    self.dropout_dep = tf.keras.layers.Dropout(params['dropout_rate'])
    self.dropout_head = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear_head_dep_dot = tf.keras.layers.Dot(axes=2)

  
  def call(self, inputs, training=False):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    
    #subwordvecs = tf.nn.embedding_lookup(bpemb_pl.vectors, inputs)
    #inputs_masked = self.masking(inputs) # not working!

    subwordvecs = self.embeddings(inputs)

    input_dropouted = self.input_dropout(subwordvecs, training=training)
    x = self.bidir_rnn(input_dropouted)
    
    x_pos = self.out_linear_pos(x)
    # after out_linear_pos x_pos.shape: (16, 32, 16)

    dep_arc_emb = self.dropout_dep( self.linear_dep_arc(x), training=training)
    head_arc_emb = self.dropout_head( self.linear_head_arc(x), training=training)
    x_heads = self.out_linear_head_dep_dot([dep_arc_emb, head_arc_emb])
    # x_heads.shape: (16, 24, 24)

    return (x_pos, x_heads)

In [ ]:
embedding_matrix = bpemb_pl.vectors

model = Model(params, embedding_matrix)
model.build(input_shape=(None ,params["sentence_len"],)) # None is batch

print("VARIABLES")

pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.9)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0
history_f1 = []
best_f1 = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for (tokenids, (upostags, heads, deprels)) in dataset_UD(is_training=True, params=params, bpemb_pl=bpemb_pl, le_upostag=le_upostag, le_deprel=le_deprel):
    with tf.GradientTape() as tape:

      y_pos, y_heads = model(tokenids, training=True) # y_deprels

      sent_lengths = tf.math.count_nonzero(tf.greater_equal(upostags, 0),axis=1)
      sent_lengths_weight_mask = tf.sequence_mask(sent_lengths, dtype=tf.float32)
      
      # zero here becomes ambiguous, it is also a class in labeldecoder
      # ideally -1 would be used for masking, however loss functions do not allow -1 labels
      upostags_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), upostags)
      #loss_upostags = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=upostags, logits=y_pos)
      loss_upostags = tfa.seq2seq.sequence_loss(logits=y_pos, targets=upostags_masked, weights = sent_lengths_weight_mask)
      loss_upostags = tf.reduce_mean(loss_upostags)

      heads_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), heads)
      loss_heads = tf.keras.losses.sparse_categorical_crossentropy(y_pred=y_heads, y_true=heads_masked, from_logits=True)
      loss_heads = tf.reduce_mean(loss_heads)

      loss = loss_upostags + loss_heads
    
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | loss_upostags: {:.4f} | Loss_heads: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), loss_upostags.numpy().item(), loss_heads.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1

  # EVALUATION
  pos_true = [] ; pos_pred = []
  heads_true = [] ; heads_pred = []
  deprels_true = [] ; deprels_pred = []
  
  for (tokenids, (upostags, heads, deprels)) in dataset_UD(is_training=False, params=params, bpemb_pl=bpemb_pl, le_upostag=le_upostag, le_deprel=le_deprel):
    
    y_pos, y_heads = model(tokenids, training=False)
    y_pos = tf.argmax(y_pos, -1)
    y_heads = tf.argmax(y_heads, -1)
    
    pos_true += upostags.numpy().flatten().tolist()
    pos_pred += y_pos.numpy().flatten().tolist()
    heads_true += heads.numpy().flatten().tolist()
    heads_pred += y_heads.numpy().flatten().tolist()
    
    
  f1_pos   = f1_score(y_true = pos_true,
                      y_pred = pos_pred,
                      labels = list( le_upostag.classes_ ),
                      sample_weight = np.sign(pos_true),
                      average='micro',)
  history_f1.append(f1_pos)

  logger.info('\n'+classification_report(y_true = pos_true,
                                         y_pred = pos_pred,
                                         labels = list( le_upostag.transform(le_upostag.classes_) ),  
                                         target_names = list(le_upostag.classes_),
                                         digits=3))

  # report dependency parser's UAS:
  tp_hits = 0
  for h_pred, h_true in zip(heads_pred, heads_true):
    if h_true >= 0 and h_pred==h_true: tp_hits += 1
  print("UAS: %.3f" % (float(tp_hits)/float(len(heads_true))) )
  
  # report dependency parser's LAS:
  #tp_hits = 0
  #for d_pred, d_true in zip(deprels_pred, deprels_true):
  #  if d_true >= 0 and d_pred==d_true: tp_hits += 1
  #print("LAS: %.3f" % (float(tp_hits)/float(len(deprels_true))) )


  if f1_pos > best_f1:
    best_f1 = f1_pos
    # you can save model here
  logger.info("Best POS F1: {:.3f}".format(best_f1))
  
  if len(history_f1) > params['num_patience'] and is_descending(history_f1):
    logger.info("Testing POS Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break


VARIABLES
[('bidirectional_2/forward_lstm_2/lstm_cell_7/kernel:0',
  TensorShape([100, 400])),
 ('bidirectional_2/forward_lstm_2/lstm_cell_7/recurrent_kernel:0',
  TensorShape([100, 400])),
 ('bidirectional_2/forward_lstm_2/lstm_cell_7/bias:0', TensorShape([400])),
 ('bidirectional_2/backward_lstm_2/lstm_cell_8/kernel:0',
  TensorShape([100, 400])),
 ('bidirectional_2/backward_lstm_2/lstm_cell_8/recurrent_kernel:0',
  TensorShape([100, 400])),
 ('bidirectional_2/backward_lstm_2/lstm_cell_8/bias:0', TensorShape([400])),
 ('output_pos/kernel:0', TensorShape([200, 18])),
 ('output_pos/bias:0', TensorShape([18])),
 ('dep_arc/kernel:0', TensorShape([200, 69])),
 ('dep_arc/bias:0', TensorShape([69])),
 ('head_arc/kernel:0', TensorShape([200, 69])),
 ('head_arc/bias:0', TensorShape([69]))]
Reading /content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu
INFO:tensorflow:Step 0 | Loss: 5.8854 | loss_upostags: 2.8917 | Loss_heads: 2.9937 | Spent: 4.0 secs | LR: 0.000300
INFO:tensorflow

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Reading /content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu
INFO:tensorflow:Step 1150 | Loss: 2.8482 | loss_upostags: 1.9068 | Loss_heads: 0.9414 | Spent: 12.7 secs | LR: 0.000266
INFO:tensorflow:Step 1200 | Loss: 2.7150 | loss_upostags: 1.9621 | Loss_heads: 0.7530 | Spent: 2.6 secs | LR: 0.000264
INFO:tensorflow:Step 1250 | Loss: 3.0294 | loss_upostags: 1.6950 | Loss_heads: 1.3344 | Spent: 2.8 secs | LR: 0.000263
INFO:tensorflow:Step 1300 | Loss: 2.9280 | loss_upostags: 1.7769 | Loss_heads: 1.1511 | Spent: 3.2 secs | LR: 0.000262
INFO:tensorflow:Step 1350 | Loss: 2.6997 | loss_upostags: 1.9411 | Loss_heads: 0.7586 | Spent: 2.8 secs | LR: 0.000260
INFO:tensorflow:Step 1400 | Loss: 2.7056 | loss_upostags: 1.7733 | Loss_heads: 0.9323 | Spent: 3.0 secs | LR: 0.000259
INFO:tensorflow:Step 1450 | Loss: 2.9321 | loss_upostags: 1.6000 | Loss_heads: 1.3321 | Spent: 2.9 secs | LR: 0.000257
INFO:tensorflow:Step 1500 | Loss: 2.6424 | loss_upostags: 1.8524 | Loss_heads: 0.7900 | Spe